https://github.com/nn116003/torchtext-tutorial/blob/master/%E8%87%AA%E5%88%86%E3%81%AE%E3%83%87%E3%83%BC%E3%82%BF%E3%81%A7%E3%82%B5%E3%82%AF%E3%83%83%E3%81%A8attention%E3%81%A4%E3%81%8D%E6%96%87%E6%9B%B8%E5%88%86%E9%A1%9E%E3%83%A2%E3%83%87%E3%83%AB%E3%82%92%E3%81%9F%E3%82%81%E3%81%99.ipynb

In [1]:
import janome
from janome.tokenizer import Tokenizer

import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.vocab import FastText

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
from torch.autograd import Variable

torch.manual_seed(0)
torch.cuda.manual_seed(0)

import inspect

In [2]:
j_t = Tokenizer()
def tokenizer(text): 
    return [tok for tok in j_t.tokenize(text, wakati=True)]

TEXT = data.Field(sequential=True, tokenize=tokenizer, lower=True, include_lengths=True, batch_first=True)
LABEL = data.Field(sequential=False, use_vocab=False)

In [4]:
# train, val, test = data.TabularDataset.splits(
#         path='./Data/', train='train.tsv',
#         validation='valid.tsv', test='test.tsv', format='tsv',
#         fields=[('Text', TEXT), ('Label', LABEL)])
train, val, test = data.TabularDataset.splits(
    path='./Data/', 
    train='train_small.tsv',
    validation='valid_small.tsv', 
    test='test_small.tsv',
    format='tsv',
    fields = [('Text', TEXT), ('Label', LABEL)])

In [5]:
print('len(train)', len(train))
print('vars(train[0])', vars(train[0]))

print('len(val)', len(val))
print('vars(val[0])', vars(val[0]))

print('len(test)', len(test))
print('vars(test[0])', vars(test[0]))

len(train) 1045
vars(train[0]) {'Text': ['text'], 'Label': 'Label'}
len(val) 350
vars(val[0]) {'Text': ['text'], 'Label': 'Label'}
len(test) 350
vars(test[0]) {'Text': ['text'], 'Label': 'Label'}


In [6]:
TEXT.build_vocab(train, min_freq=10)

FastTextの学習済みデータを利用する場合

In [ ]:
# TEXT.build_vocab(train, vectors = FastText(language="ja"), min_freq = 10)

In [8]:
train_iter, val_iter, test_iter = data.Iterator.splits(
        (train, val, test), batch_sizes=(32, 256, 256), device="cpu", repeat=False, sort=False)

In [ ]:
batch = next(iter(train_iter))
print(batch.Text)
print(batch.Label)

In [ ]:
print(batch.Text)

In [ ]:
# class EncoderRNN(nn.Module):
#     def __init__(self, emb_dim, h_dim, v_size, gpu=True, batch_first=True):
#         super(EncoderRNN, self).__init__()
#         self.gpu = gpu
#         self.h_dim = h_dim
#         self.embed = nn.Embedding(v_size, emb_dim)
#         self.embed.weight.data.copy_(TEXT.vocab.vectors)
#         self.lstm = nn.LSTM(emb_dim, h_dim, batch_first=batch_first,
#                             bidirectional=True)

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, emb_dim, h_dim, v_size, gpu=True, v_vec=None, batch_first=True):
        super(EncoderRNN, self).__init__()
        self.gpu = gpu
        self.h_dim = h_dim
        self.embed = nn.Embedding(v_size, emb_dim)
        if v_vec is not None:
            self.embed.weight.data.copy_(v_vec)
        self.lstm = nn.LSTM(emb_dim, h_dim, batch_first=batch_first,
                            bidirectional=True)

    def init_hidden(self, b_size):
        h0 = Variable(torch.zeros(1*2, b_size, self.h_dim))
        c0 = Variable(torch.zeros(1*2, b_size, self.h_dim))
        if self.gpu:
            h0 = h0.cuda()
            c0 = c0.cuda()
        return (h0, c0)

    def forward(self, sentence, lengths=None):
        self.hidden = self.init_hidden(sentence.size(0))
        emb = self.embed(sentence)
        packed_emb = emb

        if lengths is not None:
            lengths = lengths.view(-1).tolist()
            packed_emb = nn.utils.rnn.pack_padded_sequence(emb, lengths)

        out, hidden = self.lstm(packed_emb, self.hidden)

        if lengths is not None:
            out = nn.utils.rnn.pad_packed_sequence(output)[0]

        out = out[:, :, :self.h_dim] + out[:, :, self.h_dim:]

        return out

In [ ]:
class Attn(nn.Module):
    def __init__(self, h_dim):
        super(Attn, self).__init__()
        self.h_dim = h_dim
        self.main = nn.Sequential(
            nn.Linear(h_dim, 24),
            nn.ReLU(True),
            nn.Linear(24,1)
        )

    def forward(self, encoder_outputs):
        b_size = encoder_outputs.size(0)
        attn_ene = self.main(encoder_outputs.view(-1, self.h_dim)) # (b, s, h) -> (b * s, 1)
        return F.softmax(attn_ene.view(b_size, -1), dim=1).unsqueeze(2) # (b*s, 1) -> (b, s, 1)

In [ ]:
class AttnClassifier(nn.Module):
    def __init__(self, h_dim, c_num):
        super(AttnClassifier, self).__init__()
        self.attn = Attn(h_dim)
        self.main = nn.Linear(h_dim, c_num)


    def forward(self, encoder_outputs):
        attns = self.attn(encoder_outputs) #(b, s, 1)
        feats = (encoder_outputs * attns).sum(dim=1) # (b, s, h) -> (b, h)
        return F.log_softmax(self.main(feats),dim=1), attns

In [ ]:
def train_model(epoch, train_iter, optimizer, log_interval=1, batch_size=2):
    encoder.train()
    classifier.train()
    correct = 0
    for idx, batch in enumerate(train_iter):
        (x, x_l), y = batch.Text, batch.Label
        optimizer.zero_grad()
        encoder_outputs = encoder(x)
        output, attn = classifier(encoder_outputs)
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(y.data.view_as(pred)).cpu().sum()
        if idx % log_interval == 0:
            print('train epoch: {} [{}/{}], acc:{}, loss:{}'.format(
            epoch, (idx+1)*len(x), len(train_iter)*batch_size,
            correct/float(log_interval * len(x)),
            loss.data[0]))
            correct = 0

            
def test_model(epoch, test_iter):
    encoder.eval()
    classifier.eval()
    correct = 0
    for idx, batch in enumerate(test_iter):
        (x, x_l), y = batch.Text, batch.Label
        encoder_outputs = encoder(x)
        output, attn = classifier(encoder_outputs)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(y.data.view_as(pred)).cpu().sum()
    print('test epoch:{}, acc:{}'.format(epoch, correct/float(len(test))))

In [ ]:
emb_dim = 300 #単語埋め込み次元
h_dim = 3 #lstmの隠れ層の次元
class_num = 2 #予測クラウ数
lr = 0.001 #学習係数
epochs = 50 #エポック数

 # make model
encoder = EncoderRNN(emb_dim, h_dim, len(TEXT.vocab),gpu=False, v_vec = TEXT.vocab.vectors)
classifier = AttnClassifier(h_dim, class_num)

# init model
def weights_init(m):
    classname = m.__class__.__name__
    if hasattr(m, 'weight') and (classname.find('Embedding') == -1):
        nn.init.xavier_uniform(m.weight.data, gain=nn.init.calculate_gain('relu'))

for m in encoder.modules():
    print(m.__class__.__name__)
    weights_init(m)
    
for m in classifier.modules():
    print(m.__class__.__name__)
    weights_init(m)

# optim
from itertools import chain
optimizer = optim.Adam(chain(encoder.parameters(),classifier.parameters()), lr=lr)

In [ ]:
# train model
for epoch in range(epochs):
    train_model(epoch + 1, train_iter, optimizer)
    test_model(epoch + 1, val_iter)

In [ ]:
train.split(0.1)

In [ ]:
encoder.train

In [ ]:
len(TEXT.vocab)

In [ ]:
h_dim

In [ ]:
emb_dim

In [ ]:
TEXT.vocab.vectors.shape